In [7]:
import numpy as np
from sklearn.metrics import confusion_matrix

def quadratic_weighted_kappa(holistic_scores, true_scores, bins=None):
    """
    두 평가자 간의 Quadratic Weighted Kappa 점수를 계산합니다.
    실수 값을 처리할 수 있으며, bins 파라미터를 이용해 구간 설정이 가능합니다.
    
    매개변수:
    holistic_scores -- 첫 번째 평가자의 점수 리스트 (정수 또는 실수)
    true_scores -- 두 번째 평가자의 점수 리스트 (정수 또는 실수)
    bins -- 점수를 분류할 구간 경계(실수 사용 시). None일 경우 고유값 사용
    
    반환값:
    qwk -- Quadratic Weighted Kappa 점수 (float)
    """
    # 입력값을 numpy 배열로 변환 (실수값 허용)
    holistic_scores = np.array(holistic_scores, dtype=float)
    true_scores = np.array(true_scores, dtype=float)
    
    # 실수값 처리를 위한 구간화(binning) 처리
    if bins is None:
        # 고유값을 모두 추출하여 정렬
        unique_ratings = np.sort(np.unique(np.concatenate([holistic_scores, true_scores])))
        num_ratings = len(unique_ratings)
        
        # 값을 인덱스로 변환하는 매핑 생성
        rating_to_idx = {rating: idx for idx, rating in enumerate(unique_ratings)}
        
        # 점수를 인덱스로 변환
        holistic_indices = np.array([rating_to_idx[score] for score in holistic_scores])
        true_indices = np.array([rating_to_idx[score] for score in true_scores])
    else:
        # 주어진 bins로 점수를 구간화
        bins = np.array(bins)
        holistic_indices = np.digitize(holistic_scores, bins) 
        true_indices = np.digitize(true_scores, bins)
        num_ratings = len(bins) + 1
    
    # 실제 관찰된 빈도 행렬 (confusion matrix) 계산
    observed = confusion_matrix(true_indices, holistic_indices, 
                               labels=list(range(num_ratings)))
    
    # 기대 빈도 행렬 계산
    # 각 평가자의 점수 분포에 기반한 기대 빈도
    hist_true = np.bincount(true_indices, minlength=num_ratings)
    hist_holistic = np.bincount(holistic_indices, minlength=num_ratings)
    
    # 외적으로 기대 빈도 행렬 계산
    expected = np.outer(hist_true, hist_holistic) / float(len(true_scores))
    
    # 가중치 행렬 계산 (quadratic weights)
    weights = np.zeros((num_ratings, num_ratings))
    for i in range(num_ratings):
        for j in range(num_ratings):
            weights[i, j] = ((i - j) ** 2) / ((num_ratings - 1) ** 2)
    
    # QWK 계산
    numerator = np.sum(weights * observed)
    denominator = np.sum(weights * expected)
    
    # 분모가 0인 경우 (완벽한 일치)
    if denominator == 0:
        return 1.0
    
    qwk = 1 - (numerator / denominator)
    return qwk

# 사용 예시
if __name__ == "__main__":
    # 정수 예시 데이터
    holistic_scores_int = [1, 2, 3, 4, 5, 4, 3, 2, 1]
    true_scores_int = [1, 2, 3, 4, 5, 5, 4, 3, 2]
    
    qwk_int = quadratic_weighted_kappa(holistic_scores_int, true_scores_int)
    print(f"Integer Scores - Quadratic Weighted Kappa: {qwk_int:.4f}")
    
    # 실수 예시 데이터
    holistic_scores_float = [4.0]
    true_scores_float = [3.5]
    
    # 방법 1: 고유값 사용 (기본값)
    qwk_float1 = quadratic_weighted_kappa(holistic_scores_float, true_scores_float)
    print(f"Float Scores (Unique values) - Quadratic Weighted Kappa: {qwk_float1:.4f}")
    
    # 방법 2: 구간 경계 지정
    bins = [1.0, 2.0, 3.0, 4.0, 5.0]  # 1-5 사이 1점 간격 구간
    qwk_float2 = quadratic_weighted_kappa(holistic_scores_float, true_scores_float, bins)
    print(f"Float Scores (Custom bins) - Quadratic Weighted Kappa: {qwk_float2:.4f}")

Integer Scores - Quadratic Weighted Kappa: 0.8784
Float Scores (Unique values) - Quadratic Weighted Kappa: 0.0000
Float Scores (Custom bins) - Quadratic Weighted Kappa: 0.0000


In [4]:
None == False

False

In [6]:
if "":
    print("Dd")
else:
    print("ddddd")

ddddd
